In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

c:\Users\youss\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
base=''
classes = ['empty', 'not_empty'] 

In [3]:
def load_images(base,classes, img_size=(70, 30)):
    images = []
    labels = []
    
    for class_label in classes:
        class_dir = os.path.join(base, class_label)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)  # Resize image to a fixed size
            images.append(img)
            labels.append(classes.index(class_label))
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels




In [4]:
images, labels = load_images(base, classes)

images = images / 255.0

labels = to_categorical(labels, num_classes=len(classes))

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
train_indices = np.random.permutation(len(X_train))
test_indices = np.random.permutation(len(X_test))

# Shuffle x_train and y_train using the same shuffled indices
X_train = X_train[train_indices]
y_train = y_train[train_indices]

# Shuffle x_test and y_test using the same shuffled indices
X_test = X_test[test_indices]
y_test = y_test[test_indices]

In [5]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(30,70,3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))

model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
# Dense layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
122/122 [==============================] - 48s 367ms/step - loss: 0.2250 - accuracy: 0.9038 - val_loss: 0.0767 - val_accuracy: 0.9774
Epoch 2/10
122/122 [==============================] - 45s 365ms/step - loss: 0.1054 - accuracy: 0.9723 - val_loss: 0.0646 - val_accuracy: 0.9867
Epoch 3/10
122/122 [==============================] - 43s 352ms/step - loss: 0.0868 - accuracy: 0.9761 - val_loss: 0.0796 - val_accuracy: 0.9764
Epoch 4/10
122/122 [==============================] - 42s 346ms/step - loss: 0.0837 - accuracy: 0.9787 - val_loss: 0.0344 - val_accuracy: 0.9918
Epoch 5/10
122/122 [==============================] - 43s 351ms/step - loss: 0.0555 - accuracy: 0.9843 - val_loss: 0.0238 - val_accuracy: 0.9908
Epoch 6/10
122/122 [==============================] - 43s 353ms/step - loss: 0.0651 - accuracy: 0.9823 - val_loss: 0.0362 - val_accuracy: 0.9877
Epoch 7/10
122/122 [==============================] - 43s 351ms/step - loss: 0.0965 - accuracy: 0.9751 - val_loss: 0.2817 - val_ac

In [6]:
model.evaluate(X_test,y_test)

39/39 [==============================] - 1s 36ms/step - loss: 0.0041 - accuracy: 0.9984


[0.004080541897565126, 0.9983579516410828]

In [7]:
mask_image_path = 'mask_1920_1080.png'
mask_img = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

target_image_path = 'parking-1920-1080.jpg'
target_img = cv2.imread(target_image_path)


In [8]:
# Threshold the mask image to get a binary image
_, binary_mask = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)

# Find contours in the binary mask
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [9]:
output_dir = 'extracted_parts'
os.makedirs(output_dir, exist_ok=True)

# Extract and save the parts
for i, contour in enumerate(contours):
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Extract the region of interest (ROI) from the target image
    roi = target_img[y:y+h, x:x+w]
    # Save the extracted part
    part_path = os.path.join(output_dir, f'part_{i}.png')
    roi_resized = cv2.resize(roi, (70, 30))
    cv2.imwrite(part_path, roi_resized)

In [10]:
# Initialize a list to store the extracted parts
extracted_parts_array = np.empty((0, 30, 70, 3), dtype=np.uint8)  

# Extract and store the parts
for contour in contours:
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Extract the region of interest (ROI) from the target image
    roi = target_img[y:y+h, x:x+w]
    
    # Resize ROI if needed to match expected size (e.g., 64x64 pixels)
    roi_resized = cv2.resize(roi, (70, 30))/255.0
    # Append the ROI to the NumPy array
    extracted_parts_array = np.append(extracted_parts_array, [roi_resized], axis=0)


In [11]:
predictions=model.predict(extracted_parts_array)
predicted_labels = np.argmax(predictions, axis=1)

# Count occurrences of each predicted label
label_counts = np.bincount(predicted_labels)

# Print label counts
print("Label Counts:", label_counts)

# Optionally, if you want to map indices to actual labels:
class_labels = ['Empty', 'Not Empty']
label_counts_with_labels = {class_labels[i]: count for i, count in enumerate(label_counts)}
print("Label Counts with Labels:", label_counts_with_labels)

13/13 [==============================] - 1s 34ms/step
Label Counts: [122 274]
Label Counts with Labels: {'Empty': 122, 'Not Empty': 274}


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 30, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 14, 96)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 14, 256)        614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 1, 6, 256)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 6, 384)         885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 6, 384)         1

In [44]:
import onnxruntime as ort
import asyncio
cap = cv2.VideoCapture('parking_1920_1080.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (1920, 1080))
ort_session = ort.InferenceSession('model.onnx')
frame_skip = 10  # Number of frames to skip
frame_count = 0
previous_prediction = None
def predict_on_frame(frame):
    extracted_parts_array = np.empty((0, 30, 70, 3), dtype=np.uint8)  

    for contour in contours:
        
        x, y, w, h = cv2.boundingRect(contour)
        target_img=frame
        
        roi = target_img[y:y+h, x:x+w]
        roi = roi.astype(np.float32) 
        
        roi_resized = cv2.resize(roi, (70, 30))/255.0
        
        extracted_parts_array = np.append(extracted_parts_array, [roi_resized], axis=0)
    input_name = ort_session.get_inputs()[0].name
    result = ort_session.run(None, {input_name: extracted_parts_array})
    predicted_labels = np.argmax(result[0], axis=1)
    label_counts = np.bincount(predicted_labels)
    return label_counts
    
    
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret:
        if frame_count % frame_skip == 0:
            label_counts=predict_on_frame(frame)
            previous_prediction=label_counts
        else:
            label_counts=previous_prediction

        cv2.putText(frame, f'Empty Spaces: {label_counts[0]}', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        
        
        
        out.write(frame)
        frame_count += 1
        
    else:
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()

In [21]:
import tf2onnx
import onnx
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, 'model.onnx')


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   -------------------------------------- 101.8/101.8 kB 234.1 kB/s eta 0:00:00
